In [1]:
data = [[]]
min_sup = 0.3
min_conf = 1
ls_max_itemset = list()
ls_assoc = list()

In [2]:
def is_in_set(setA, setB):
    for item in setA:
        if item not in setB:
            return False
    return True

In [3]:
def freq_of_set(setA):
    freq_set = 0
    for itemset in data:
        freq_item = 0
        for item in setA:
            if item in itemset:
                freq_item += 1
        if freq_item == len(setA):
            freq_set += 1
    return freq_set

In [4]:
def is_satisf_minsup(setA):
    return freq_of_set(setA)/len(data) >= min_sup

In [5]:
def can_make_newset(setA, setB):
    if not is_satisf_minsup(setA) or not is_satisf_minsup(setB):
        return None
    
    num_item = 0
    add_item = list()
    for item in setB:
        if item in setA:
            num_item += 1
        else:
            add_item.append(item)
    if add_item.__len__() == 1 and is_satisf_minsup(setA + add_item):
        return setA + add_item
    return None

In [6]:
def is_set_in_ls(setA, ls_itemset):
    for itemset in ls_itemset:
        if is_in_set(setA, itemset):
            return True
    return False

In [7]:
def generation(ls_itemset):
    new_ls_itemset = list()
    for itemsetA in ls_itemset:
        for itemsetB in ls_itemset[ls_itemset.index(itemsetA):]:
            new_set = can_make_newset(itemsetA, itemsetB) 
            if new_set != None and not is_set_in_ls(new_set, new_ls_itemset):
                new_ls_itemset.append(new_set)
    return new_ls_itemset

In [8]:
def load_initial():
    first_set = list()
    for itemset in data:
        for item in itemset:
            if [item,] not in first_set and is_satisf_minsup([item,]):
                first_set.append([item, ])
    return first_set

In [9]:
def clear_old_set(ls_itemset):
    new_ls_max_itemset = list()
    for old_itemset in ls_max_itemset:
        if not is_set_in_ls(old_itemset, ls_itemset):
            new_ls_max_itemset.append(old_itemset)
    return new_ls_max_itemset + ls_itemset

In [10]:
def exc_cal_ls_max_itemset():
    global ls_max_itemset
    ls_itemset = load_initial()
    ls_max_itemset = ls_itemset
    
    while ls_itemset.__len__() > 0:
        print(ls_itemset)
        ls_itemset = generation(ls_itemset)
        ls_max_itemset = clear_old_set(ls_itemset)

In [11]:
from itertools import combinations 
def get_ls_item(itemset, num_item):
    return [[j for j in i ] for i in combinations(itemset, num_item)]

In [12]:
def get_ls_assoc():
    for itemset in ls_max_itemset:
        for i in range(1, len(itemset)):
            ls_src = get_ls_item(itemset, i)
            for j in range(1, len(itemset) - i + 1):
                ls_des = get_ls_item(itemset, j)
                make_assoc(ls_src, ls_des, ls_assoc)

In [13]:
def make_assoc(ls_src, ls_des, ls_assoc):    
    for item_src in ls_src:
        for item_des in ls_des:
            if can_make_assoc(item_src, item_des):
                ls_assoc.append(','.join(item_src) + '->' + ','.join(item_des))

In [14]:
def can_make_assoc(p, q):
    return not is_in_set(p, q) and not is_in_set(q, p) and freq_of_set(p + q)/freq_of_set(p) >= min_conf

In [15]:
def load_data(filename):
    return [[j for j in i.split(', ')] for i in open(filename, 'r').read().split('\n')]

In [16]:
def load_test():
    '''
        input: ...
    '''
    return 'i2,i1'

In [17]:
def test():
    itemset_test = load_test().split(',')
    for itemset in ls_assoc:
        item_src, item_des = itemset.split('->')
        item_src = item_src.split(',')
        if is_in_set(item_src, itemset_test) and len(item_src) == len(itemset_test):
            print(item_des)

In [18]:
def main():
    global data
    
    data = load_data('data.txt')
    exc_cal_ls_max_itemset()
    get_ls_assoc()
    test()
    
if __name__ == '__main__':
    main()

[['i1'], ['i7'], ['i2'], ['i6'], ['i8'], ['i4']]
[['i1', 'i7'], ['i1', 'i2'], ['i1', 'i6'], ['i2', 'i6']]
[['i1', 'i2', 'i6']]
i6


In [19]:
ls_max_itemset

[['i8'], ['i4'], ['i1', 'i7'], ['i1', 'i2', 'i6']]

In [22]:
ls_assoc

['i7->i1',
 'i2->i1',
 'i2->i6',
 'i2->i1,i6',
 'i1,i2->i6',
 'i1,i6->i2',
 'i2,i6->i1']